In [ ]:
import pandas as pd
import numpy as np
import glob
import os

In [ ]:
mp2 = pd.read_csv("/data/dzeiberg/ppi/y2hEdgotyping/mutpred2Results_v2/variants.faa.out")

In [ ]:
mp2 = mp2.assign(mm=mp2["Molecular mechanisms with Pr >= 0.01 and P < 1.00"])

In [ ]:
import re
def getFloats(s):
    numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
    rx = re.compile(numeric_const_pattern,re.VERBOSE)
    return [float(f) for f in rx.findall(s)]

In [ ]:
def getStability(res,prob=True):
    property_pr_p = [(si[:si.find("(")],getFloats(si))for si in sorted([s.strip() for s in res.split(";")])]
    for (prop,vals) in property_pr_p:
        pr,p = vals[-2:]
        if prop == "Altered Stability ":
            if prob:
                return pr
            return p

In [ ]:
import networkx as nx
edgotype_train = nx.read_gpickle("data/y2hEdgotyping/edgotype_train.gpickle")

In [ ]:
e = next(iter(edgotype_train.edges(data=True)))

In [ ]:
def make_y2h_target(d):
    names = ["LWH1_f_","LWH10_f_","LWH25_f_",
             "LWA_f_","LWAH1_f_"]
    deltas = np.zeros(len(names))
    for i,name in enumerate(names):
        deltas[i] = d[name+"wt"] - d[name+"mt"]
    return np.any(deltas >= 2)

In [ ]:
probs,labels = list(zip(*[(getStability(e[2]["Molecular mechanisms with Pr >= 0.01 and P < 0.99"]),
                                    make_y2h_target(e[2])) for e in edgotype_train.edges(data=True)]))



In [ ]:
sum([p is None for p in probs])

In [ ]:
len(probs)

In [ ]:
labels = np.array([labels[i] for i in range(len(labels)) if probs[i] is not None])
probs = np.array([p for p in probs if p is not None])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(probs[labels],density=True)
plt.hist(probs[~labels],alpha=.5,density=True)

In [ ]:
from scipy.io import loadmat

In [ ]:
m = loadmat("/data/utilities/bio/mutpred2_source/all_models/model_Stability_081416.mat")

In [ ]:
m.keys()

In [ ]:
m["nn_model"].shape

In [ ]:
m["features"].shape